In [1]:
!pip install huggingface

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


Using a pre-existing pipeline

In [24]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

classifier(
    [
        "This is so simple and concise",
        "Transformers are very hard to tune"
    ]
    )

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9996987581253052},
 {'label': 'NEGATIVE', 'score': 0.9991831183433533}]

# Manual Process

**Step 1** : Use an autotokenizer to convert text to numerical values

- This is called as tokenization and we use tokenizer to do this.

- A tokenizer is initiated with a checkpoint of pretrained model

In [44]:
from transformers import AutoTokenizer

# initiate checkpoints to specify which model weights we are going to use
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [45]:
raw_inputs=["Hello, this is awesome","I will skip this this time"]

inputs = tokenizer(raw_inputs,padding = True, truncation= True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[  101,  7592,  1010,  2023,  2003, 12476,   102,     0],
       [  101,  1045,  2097, 13558,  2023,  2023,  2051,   102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


**Step 2**: create a Model

In [46]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [47]:
outputs = model(inputs)

In [48]:
print(outputs)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-4.2142158,  4.572801 ],
       [ 4.4170356, -3.5259855]], dtype=float32)>, hidden_states=None, attentions=None)


In [49]:
print(outputs.logits)

tf.Tensor(
[[-4.2142158  4.572801 ]
 [ 4.4170356 -3.5259855]], shape=(2, 2), dtype=float32)


Step 3: To get output predictions we need to comnvert logits to accuracy prpedictions

- we use softmax function to do this
- There are many loss functions but the prediction values might change depending on the loss function we use

In [52]:
import tensorflow

predictions = tensorflow.math.softmax(outputs.logits,axis=-1)
print(predictions)

tf.Tensor(
[[1.5267951e-04 9.9984729e-01]
 [9.9964499e-01 3.5500599e-04]], shape=(2, 2), dtype=float32)


In [53]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [54]:
import numpy as np

In [57]:
Results = np.array(predictions)

In [65]:
print(Results)


[[1.5267951e-04 9.9984729e-01]
 [9.9964499e-01 3.5500599e-04]]


In [75]:
print("Probability of Sentence 1 being negative: ",np.round(Results[0][0]*100),"positive: ",np.round(Results[0][1]*100))
print("Probability of Sentence 2 being negative: ",np.round(Results[1][0]*100),"positive: ",np.round(Results[1][1]*100))

Probability of Sentence 1 being negative:  0.0 positive:  100.0
Probability of Sentence 2 being negative:  100.0 positive:  0.0
